# Support Vector Machines-1: Theory, Visualization, and Implementation
This notebook covers the mathematical foundation of linear SVMs, the kernel trick, support vectors, hyperplanes, margin concepts, and includes both scikit-learn and from-scratch implementations using the Iris dataset.

## Q1. What is the mathematical formula for a linear SVM?

A linear SVM finds the hyperplane that best separates two classes:

w·x + b = 0

where w is the weight vector, x is the input vector, and b is the bias.

## Q2. What is the objective function of a linear SVM?

The objective is to maximize the margin while minimizing classification error:

Minimize: (1/2)||w||² + C Σ ξᵢ
Subject to: yᵢ(w·xᵢ + b) ≥ 1 - ξᵢ, ξᵢ ≥ 0

where ξᵢ are slack variables and C is the regularization parameter.

## Q3. What is the kernel trick in SVM?

The kernel trick allows SVMs to perform classification in higher-dimensional spaces without explicitly computing the coordinates. It uses a kernel function (e.g., RBF, polynomial) to compute the inner product in the transformed space, enabling non-linear decision boundaries.

## Q4. What is the role of support vectors in SVM? Explain with example.

Support vectors are the data points closest to the hyperplane. They define the margin and the position of the hyperplane. Only these points affect the model; removing non-support vectors does not change the decision boundary.

In [ ]:
# Example: Visualizing support vectors
from sklearn import datasets
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import numpy as np

iris = datasets.load_iris()
X = iris.data[:, :2]
y = (iris.target != 0) * 1
svc = SVC(kernel='linear', C=1)
svc.fit(X, y)
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', s=30)
plt.scatter(svc.support_vectors_[:, 0], svc.support_vectors_[:, 1], s=100, facecolors='none', edgecolors='k', label='Support Vectors')
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')
plt.legend()
plt.title('Support Vectors in SVM')
plt.show()

## Q5. Illustrate with examples and graphs: Hyperplane, Marginal plane, Soft margin, and Hard margin in SVM

In [ ]:
# Visualizing SVM concepts
from sklearn.datasets import make_blobs
from sklearn.svm import SVC

X, y = make_blobs(n_samples=50, centers=2, random_state=6)
svc = SVC(kernel='linear', C=1e6)
svc.fit(X, y)

plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', s=30)
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# Create grid to evaluate model
gx = np.linspace(xlim[0], xlim[1], 30)
gy = np.linspace(ylim[0], ylim[1], 30)
xx, yy = np.meshgrid(gx, gy)
xy = np.vstack([xx.ravel(), yy.ravel()]).T
Z = svc.decision_function(xy).reshape(xx.shape)

# Plot decision boundary and margins
ax.contour(xx, yy, Z, colors='k', levels=[-1, 0, 1], alpha=0.5, linestyles=['--', '-', '--'])
ax.scatter(svc.support_vectors_[:, 0], svc.support_vectors_[:, 1], s=100, linewidth=1, facecolors='none', edgecolors='k')
plt.title('Hard Margin SVM')
plt.show()

# Soft margin
svc_soft = SVC(kernel='linear', C=0.1)
svc_soft.fit(X, y)
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', s=30)
ax = plt.gca()
Z = svc_soft.decision_function(xy).reshape(xx.shape)
ax.contour(xx, yy, Z, colors='k', levels=[-1, 0, 1], alpha=0.5, linestyles=['--', '-', '--'])
ax.scatter(svc_soft.support_vectors_[:, 0], svc_soft.support_vectors_[:, 1], s=100, linewidth=1, facecolors='none', edgecolors='k')
plt.title('Soft Margin SVM')
plt.show()

## Q6. SVM Implementation through Iris dataset

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = iris.data[:, :2]
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

svc = SVC(kernel='linear', C=1)
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))

In [ ]:
# Plotting decision boundaries for two features
def plot_decision_boundary(clf, X, y):
    h = .02
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', s=30)
    plt.xlabel('Sepal length')
    plt.ylabel('Sepal width')
    plt.title('SVM Decision Boundary')
    plt.show()

plot_decision_boundary(svc, X_test, y_test)

In [ ]:
# Trying different values of C
for C in [0.01, 0.1, 1, 10, 100]:
    svc = SVC(kernel='linear', C=C)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f'C={C}: Accuracy={acc:.2f}')

## Bonus: Implement a linear SVM classifier from scratch and compare with scikit-learn

In [ ]:
# Simple linear SVM from scratch (using subgradient descent, binary classification)
class LinearSVM:
    def __init__(self, lr=0.001, lambda_param=0.01, n_iters=1000):
        self.lr = lr
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None
    def fit(self, X, y):
        n_samples, n_features = X.shape
        y_ = np.where(y <= 0, -1, 1)
        self.w = np.zeros(n_features)
        self.b = 0
        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y_[idx] * (np.dot(x_i, self.w) + self.b) >= 1
                if condition:
                    self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    self.w -= self.lr * (2 * self.lambda_param * self.w - np.dot(x_i, y_[idx]))
                    self.b -= self.lr * y_[idx]
    def predict(self, X):
        approx = np.dot(X, self.w) + self.b
        return np.sign(approx)

# Use only two classes for comparison
X_bin = X[y < 2]
y_bin = y[y < 2]
X_train_bin, X_test_bin, y_train_bin, y_test_bin = train_test_split(X_bin, y_bin, test_size=0.3, random_state=42)

svm_scratch = LinearSVM(lr=0.001, lambda_param=0.01, n_iters=1000)
svm_scratch.fit(X_train_bin, y_train_bin)
y_pred_scratch = svm_scratch.predict(X_test_bin)
acc_scratch = np.mean(y_pred_scratch == np.where(y_test_bin == 0, -1, 1))
print('From-scratch SVM accuracy:', acc_scratch)

# Compare with scikit-learn
svc_bin = SVC(kernel='linear', C=1)
svc_bin.fit(X_train_bin, y_train_bin)
y_pred_sklearn = svc_bin.predict(X_test_bin)
acc_sklearn = accuracy_score(y_test_bin, y_pred_sklearn)
print('scikit-learn SVM accuracy:', acc_sklearn)